# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
 
# Import API key
from api_keys import geoapify_key
print(geoapify_key)

ae2ad391884942d1b2430c242265e2c7


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,10.76,89,13,3.76,AU,1690058570
1,1,mutoko,-17.3970,32.2268,9.54,94,4,1.23,ZW,1690058768
2,2,bredasdorp,-34.5322,20.0403,11.09,90,100,3.17,ZA,1690058768
3,3,salavan,15.7167,106.4167,22.45,99,99,1.19,LA,1690058583
4,4,khandyga,62.6667,135.6000,13.94,90,20,1.42,RU,1690058625


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo = True, size = 'Humidity', tiles = 'OSM', frame_width = 500, frame_height=400, color = 'Max Temp')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities = city_data_df[(city_data_df['Humidity']<70)&(city_data_df['Cloudiness']>20)]
# Drop any rows with null values
my_cities.dropna()

# Display sample data
my_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,edd,13.9297,41.6925,33.47,64,95,2.69,ER,1690058769
20,20,djibo,14.1022,-1.6306,29.09,61,81,7.84,BF,1690058770
31,31,tidjikja,18.5564,-11.4271,34.64,31,40,5.20,MR,1690058772
36,36,adamstown,-25.0660,-130.1015,20.80,67,21,7.99,PN,1690058774
41,41,el porvenir,13.7617,-87.3458,27.63,46,99,2.53,HN,1690058775


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities.copy()[['City', 'Country', 'Lat', 'Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,edd,ER,13.9297,41.6925,64,
20,djibo,BF,14.1022,-1.6306,61,
31,tidjikja,MR,18.5564,-11.4271,31,
36,adamstown,PN,-25.0660,-130.1015,67,
41,el porvenir,HN,13.7617,-87.3458,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':"accommodation.hotel",
    'apiKey': geoapify_key,
    'limit':10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edd - nearest hotel: No hotel found
djibo - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
aliwal north - nearest hotel: No hotel found
la bouilladisse - nearest hotel: Bus parada1
thompson - nearest hotel: Thompson Inn
cushing - nearest hotel: Executive Inn & Suites
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
keetmanshoop - nearest hotel: Canyon Hotel
kenora - nearest hotel: Travelodge
kalmunai - nearest hotel: Saji new food
klyuchi - nearest hotel: No hotel found
mikhaylov - nearest hotel: No hotel found
al mijlad - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas
kone - nearest hotel: Pacifik Appartels
bilibino - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
jamestown - nearest hotel: DoubleTree Jamestown
afaahiti - nearest hotel: Omati Lodge
prainha - nearest hotel: No hotel found
la r

,City,Country,Lat,Lng,Humidity,Hotel Name
12,edd,ER,13.9297,41.6925,64,No hotel found
20,djibo,BF,14.1022,-1.6306,61,No hotel found
31,tidjikja,MR,18.5564,-11.4271,31,No hotel found
36,adamstown,PN,-25.0660,-130.1015,67,No hotel found
41,el porvenir,HN,13.7617,-87.3458,46,No hotel found
...,...,...,...,...,...,...
563,karasburg,NaN,-28.0167,18.7500,15,U&K Bed & Breakfast
577,tocopilla,CL,-22.0920,-70.1979,67,hotel Isidora
582,suwayr,SA,30.1171,40.3893,13,No hotel found
585,strezhevoy,RU,60.7333,77.5889,68,"Гостиница ""Сибирячка"""


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng', 'Lat', geo = True, tiles = 'OSM', framewidth=800, frameheight = 600, size = 'Humidity', hover_cols = ['Hotel Name','Country'], color = 'City')

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)